In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import requests

pd.set_option('display.max_columns', None)

header = {'User-Agent': "mr.muffin235@gmail.com"}

companyTickers = requests.get("https://www.sec.gov/files/company_tickers.json",
                              headers=header)
#getting ticker data from sec api
companyData = pd.DataFrame.from_dict(companyTickers.json(), orient='index')
companyData = companyData.set_index('ticker') 
companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

#input ticker 
#ticker = input("Stock ticker: ")

In [28]:
# COMPANY
company = companyData.cik_str['META']
companyConcept = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{company}.json', headers=header)

In [29]:
companyConcept.json()['facts']['us-gaap'].keys()

dict_keys(['AccountsPayableAndOtherAccruedLiabilitiesCurrent', 'AccountsPayableCurrent', 'AccountsPayableOtherCurrent', 'AccountsPayableTradeCurrent', 'AccountsReceivableNetCurrent', 'AccruedIncomeTaxesCurrent', 'AccruedIncomeTaxesNoncurrent', 'AccruedLiabilitiesCurrent', 'AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', 'AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax', 'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'AdditionalPaidInCapital', 'AdjustmentsRelatedToTaxWithholdingForShareBasedCompensation', 'AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue', 'AdjustmentsToAdditionalPaidInCapitalTaxEffectFromShareBasedCompensation', 'AdjustmentToAdditionalPaidInCapitalIncomeTaxEffectFromShareBasedCompensationNet', 'AdvertisingExpense', 'AdvertisingRevenue', 'AllocatedShareBasedCompensationExpense', 'AllowanceForDoubtfulAccountsReceivableCurrent', 'AmortizationOfIntangibleAsset

In [34]:
companyConcept.json()['facts']['us-gaap']['LiabilitiesCurrent']

{'label': 'Liabilities, Current',
 'description': 'Total obligations incurred as part of normal operations that are expected to be paid during the following twelve months or within one business cycle, if longer.',
 'units': {'USD': [{'end': '2011-12-31',
    'val': 899000000,
    'accn': '0001193125-12-325997',
    'fy': 2012,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2012-07-31'},
   {'end': '2011-12-31',
    'val': 899000000,
    'accn': '0001326801-12-000006',
    'fy': 2012,
    'fp': 'Q3',
    'form': '10-Q',
    'filed': '2012-10-24'},
   {'end': '2011-12-31',
    'val': 899000000,
    'accn': '0001326801-13-000003',
    'fy': 2012,
    'fp': 'FY',
    'form': '10-K',
    'filed': '2013-02-01',
    'frame': 'CY2011Q4I'},
   {'end': '2012-06-30',
    'val': 1034000000,
    'accn': '0001193125-12-325997',
    'fy': 2012,
    'fp': 'Q2',
    'form': '10-Q',
    'filed': '2012-07-31',
    'frame': 'CY2012Q2I'},
   {'end': '2012-09-30',
    'val': 1080000000,
    'accn': '0001

In [24]:
# Assets
assets_current = companyConcept.json()['facts']['us-gaap']['AssetsCurrent']
assets = pd.DataFrame(assets_current['units']['USD'])


assets_final = assets[(assets['fp'] == 'FY') & (assets['form'] == '10-K')]
assets_final.drop_duplicates('end')

,end,val,accn,fy,fp,form,filed,frame
2,2008-12-31,6157000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
5,2009-12-31,9797000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,NaN
13,2010-12-31,13747000000,0001193125-11-016253,2010,FY,10-K,2011-01-28,NaN
21,2011-12-31,17490000000,0001193125-12-032846,2011,FY,10-K,2012-02-01,NaN
29,2012-12-31,21296000000,0001193125-13-028520,2012,FY,10-K,2013-01-30,NaN
37,2013-12-31,24625000000,0001018724-14-000006,2013,FY,10-K,2014-01-31,NaN
45,2014-12-31,31327000000,0001018724-15-000006,2014,FY,10-K,2015-01-30,NaN
53,2015-12-31,36474000000,0001018724-16-000172,2015,FY,10-K,2016-01-29,NaN
61,2016-12-31,45781000000,0001018724-17-000011,2016,FY,10-K,2017-02-10,NaN
69,2017-12-31,60197000000,0001018724-18-000005,2017,FY,10-K,2018-02-02,NaN


In [25]:
# Libilities
liabilities_list = companyConcept.json()['facts']['us-gaap']['LiabilitiesCurrent']
liabilities = pd.DataFrame(liabilities_list['units']['USD'])
liabilities_final = liabilities[(liabilities['fp'] == 'FY') & (liabilities['form'] == '10-K')]
liabilities_final.drop_duplicates('end')

,end,val,accn,fy,fp,form,filed,frame
2,2008-12-31,4746000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,CY2008Q4I
5,2009-12-31,7364000000,0001193125-10-016098,2009,FY,10-K,2010-01-29,NaN
13,2010-12-31,10372000000,0001193125-11-016253,2010,FY,10-K,2011-01-28,NaN
21,2011-12-31,14896000000,0001193125-12-032846,2011,FY,10-K,2012-02-01,NaN
29,2012-12-31,19002000000,0001193125-13-028520,2012,FY,10-K,2013-01-30,NaN
37,2013-12-31,22980000000,0001018724-14-000006,2013,FY,10-K,2014-01-31,NaN
45,2014-12-31,28089000000,0001018724-15-000006,2014,FY,10-K,2015-01-30,NaN
53,2015-12-31,33899000000,0001018724-16-000172,2015,FY,10-K,2016-01-29,NaN
61,2016-12-31,43816000000,0001018724-17-000011,2016,FY,10-K,2017-02-10,NaN
69,2017-12-31,57883000000,0001018724-18-000005,2017,FY,10-K,2018-02-02,NaN


In [5]:
# Libilities
liabilities_list = companyConcept.json()['facts']['us-gaap']['Liabilities']
liabilities = pd.DataFrame(liabilities_list['units']['USD'])
liabilities_final = liabilities[(liabilities['fp'] == 'FY') & (liabilities['form'] == '10-K')]
liabilities_final.drop_duplicates('end')

,end,val,accn,fy,fp,form,filed,frame
3,2015-12-31,27130000000,0001652044-17-000008,2016,FY,10-K,2017-02-03,CY2015Q4I
7,2016-12-31,28461000000,0001652044-17-000008,2016,FY,10-K,2017-02-03,NaN
15,2017-12-31,44793000000,0001652044-18-000007,2017,FY,10-K,2018-02-06,NaN
23,2018-12-31,55164000000,0001652044-19-000004,2018,FY,10-K,2019-02-05,NaN
31,2019-12-31,74467000000,0001652044-20-000008,2019,FY,10-K,2020-02-04,NaN
39,2020-12-31,97072000000,0001652044-21-000010,2020,FY,10-K,2021-02-03,NaN
47,2021-12-31,107633000000,0001652044-22-000019,2021,FY,10-K,2022-02-02,NaN
55,2022-12-31,109120000000,0001652044-23-000016,2022,FY,10-K,2023-02-03,CY2022Q4I


In [6]:
# Revenue
revenue_list = companyConcept.json()['facts']['us-gaap']['SalesRevenueNet']
revenue = pd.DataFrame(revenue_list['units']['USD'])
revenue_final = revenue[(revenue['fp'] == 'FY') & (revenue['form'] == '10-K')]
revenue_final.drop_duplicates('start')

KeyError: 'SalesRevenueNet'

In [10]:
# Revenue
revenue_list = companyConcept.json()['facts']['us-gaap']['Revenues']
revenue = pd.DataFrame(revenue_list['units']['USD'])

revenue_final = revenue[(revenue['fp'] == 'FY') & (revenue['form'] == '10-K')]
revenue_final.drop_duplicates('val')

,start,end,val,accn,fy,fp,form,filed,frame
0,2013-01-01,2013-12-31,55519000000,0001652044-16-000012,2015,FY,10-K,2016-02-11,NaN
4,2014-01-01,2014-12-31,66001000000,0001652044-16-000012,2015,FY,10-K,2016-02-11,NaN
14,2015-01-01,2015-12-31,74989000000,0001652044-16-000012,2015,FY,10-K,2016-02-11,NaN
28,2016-01-01,2016-12-31,90272000000,0001652044-17-000008,2016,FY,10-K,2017-02-03,NaN
40,2017-01-01,2017-12-31,110855000000,0001652044-18-000007,2017,FY,10-K,2018-02-06,CY2017
47,2018-01-01,2018-12-31,136819000000,0001652044-21-000010,2020,FY,10-K,2021-02-03,CY2018
49,2019-01-01,2019-12-31,161857000000,0001652044-21-000010,2020,FY,10-K,2021-02-03,NaN
54,2020-01-01,2020-12-31,182527000000,0001652044-21-000010,2020,FY,10-K,2021-02-03,NaN
59,2021-01-01,2021-12-31,257637000000,0001652044-22-000019,2021,FY,10-K,2022-02-02,CY2021


In [22]:
# Expenses
expenses_list = companyConcept.json()['facts']['us-gaap']['CostsAndExpenses']
expenses = pd.DataFrame(expenses_list['units']['USD'])

expenses[ & (expenses['fp'] == 'FY')]

#expense_final = expenses[(expenses['fp'] == 'FY') & (expenses['form'] == '10-K')]
#expense_final.drop_duplicates(subset=['val'])

,start,end,val,accn,fy,fp,form,filed,frame
